In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import *
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from scipy.stats import pearsonr
from collections import defaultdict
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img, ImageDataGenerator
import random
from tqdm import tqdm
import time
from concurrent.futures import ThreadPoolExecutor
import os
from sklearn.base import clone

# --- Configuração dos modelos ---
from tensorflow.keras.applications.xception import Xception, preprocess_input as preprocess_xception
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input as preprocess_vgg16
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input as preprocess_vgg19
from tensorflow.keras.applications.resnet import ResNet50, ResNet101, ResNet152, preprocess_input as preprocess_resnet
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, ResNet101V2, ResNet152V2, preprocess_input as preprocess_resnet_v2
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input as preprocess_inception_v3
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input as preprocess_inception_resnet_v2
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input as preprocess_mobilenet
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input as preprocess_mobilenet_v2
from tensorflow.keras.applications.densenet import DenseNet121, DenseNet169, DenseNet201, preprocess_input as preprocess_densenet
from tensorflow.keras.applications.nasnet import NASNetMobile, NASNetLarge, preprocess_input as preprocess_nasnet
from tensorflow.keras.applications.efficientnet import EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3, EfficientNetB4, EfficientNetB5, EfficientNetB6, EfficientNetB7, preprocess_input as preprocess_efficientnet

model_input_sizes = {model: (1024, 1024) for model in ['DenseNet169']}

model_info = {'DenseNet169': (DenseNet169, preprocess_densenet)}

# Aumentadores globais
data_augmenters = [
    ImageDataGenerator(horizontal_flip=True),
    ImageDataGenerator(brightness_range=[0.8, 1.2]),
    ImageDataGenerator(preprocessing_function=lambda x: x + np.random.normal(0, 5, x.shape))
]

# --- Funções auxiliares ---
def augment_and_extract(model, img_path, model_name):
    img = load_img(img_path, target_size=model_input_sizes[model_name])
    augmenter = random.choice(data_augmenters)
    aug_img = augment_image(img, augmenter)
    aug_img_array = img_to_array(aug_img)
    aug_img_array = np.expand_dims(aug_img_array, axis=0)
    preprocessed_aug_img = model_info[model_name][1](aug_img_array)
    feats = model.predict(preprocessed_aug_img, verbose=0)
    return feats.flatten()

def augment_image(img, augmenter):
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    aug_iter = augmenter.flow(img_array, batch_size=1)
    aug_img = next(aug_iter)[0].astype(np.uint8)
    return array_to_img(aug_img)

def extract_features_from_paths(model, img_paths, model_name):
    features = []
    for img_path in img_paths:
        img = load_img(img_path, target_size=model_input_sizes[model_name])
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        preprocessed_img = model_info[model_name][1](img_array)
        feats = model.predict(preprocessed_img, verbose=0)
        features.append(feats.flatten())
    return features

def extract_features_augmented(model, img_paths, model_name, target_len=50):
    features = extract_features_from_paths(model, img_paths, model_name)
    needed = target_len - len(features)
    if needed > 0:
        with ThreadPoolExecutor(max_workers=1) as executor:
            futures = [executor.submit(augment_and_extract, model, random.choice(img_paths), model_name) for _ in range(needed)]
            for f in futures:
                features.append(f.result())
    return features[:target_len]

def run_stratified_kfold_regressions_from_images(df, model, model_name):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    all_results = []
    all_preds_txt = []

    models = [
        ('Linear Regression', LinearRegression()),
        ('Random Forest', RandomForestRegressor(random_state=42)),
        ('SVR', SVR()),
        ('Lasso', Lasso(random_state=42)),
        ('ElasticNet', ElasticNet(random_state=42)),
        ('Ridge', Ridge(random_state=42))
    ]

    for fold, (train_idx, test_idx) in enumerate(skf.split(df, df['Group'])):
        df_train = df.iloc[train_idx]
        df_test = df.iloc[test_idx]

        id_to_train_paths = defaultdict(list)
        id_to_test_paths = defaultdict(list)

        for _, row in df_train.iterrows():
            id_to_train_paths[row['ID Animal']].append(row['Path'])
        for _, row in df_test.iterrows():
            id_to_test_paths[row['ID Animal']].append(row['Path'])

        train_features, test_features = {}, {}
        for animal_id in tqdm(id_to_train_paths.keys(), desc=f"Fold {fold+1} - Treino ({model_name})"):
            paths = id_to_train_paths[animal_id]
            feats = extract_features_augmented(model, paths, model_name, target_len=50)
            train_features[animal_id] = np.mean(feats, axis=0)

        for animal_id in tqdm(id_to_test_paths.keys(), desc=f"Fold {fold+1} - Teste ({model_name})"):
            paths = id_to_test_paths[animal_id]
            feats = extract_features_from_paths(model, paths, model_name)
            test_features[animal_id] = np.mean(feats, axis=0)

        df_weights = df[['ID Animal', 'Peso', 'Group']].drop_duplicates().set_index('ID Animal')
        train_df = pd.DataFrame.from_dict(train_features, orient='index').join(df_weights)
        test_df = pd.DataFrame.from_dict(test_features, orient='index').join(df_weights)

        X_train = train_df.iloc[:, :-2].values
        y_train = train_df['Peso'].values
        X_test = test_df.iloc[:, :-2].values
        y_test = test_df['Peso'].values

        for name, reg_model in models:
            model_clone = clone(reg_model)
            model_clone.fit(X_train, y_train)
            y_pred = model_clone.predict(X_test)

            print(f"\n--- Regressão: {name} | Modelo CNN: {model_name} ---")
            fold_metrics = []

            for i, (real, pred) in enumerate(zip(y_test, y_pred)):
                animal_id = test_df.index[i]
                all_preds_txt.append((model_name, name, fold+1, animal_id, real, pred))

            mse = mean_squared_error(y_test, y_pred)
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            r = pearsonr(y_test, y_pred)[0] if np.var(y_pred) > 0 else 0

            print(f"[Fold {fold+1}] MSE: {mse:.2f} | RMSE: {rmse:.2f} | MAE: {mae:.2f} | R2: {r2:.2f} | R: {r:.2f}")

            all_results.append({
                'REGRESSAO': name,
                'FOLD': fold + 1,
                'MSE': mse,
                'RMSE': rmse,
                'MAE': mae,
                'R2': r2,
                'R': r,
                'CNN': model_name
            })

    lines = ["CNN,REGRESSAO,FOLD,ID,PESO_REAL,PESO_PREDITO"]
    for cnn, reg, fold, id_, real, pred in all_preds_txt:
        lines.append(f"{cnn},{reg},{fold},{id_},{real:.2f},{pred:.2f}")
    with open(f"predicoes_completas_{model_name}.txt", "w", encoding="utf-8") as f:
        f.write("\n".join(lines))
    print(f"\n📝 Predições consolidadas salvas em predicoes_completas_{model_name}.txt")

    df_result = pd.DataFrame(all_results)
    for reg_name in df_result['REGRESSAO'].unique():
        df_model = df_result[df_result['REGRESSAO'] == reg_name]
        print(f"\n>>> MÉDIA DOS RESULTADOS - {reg_name}")
        for metric in ['MSE', 'RMSE', 'MAE', 'R2', 'R']:
            values = df_model[metric].values
            print(f"{metric}: {np.mean(values):.2f} ± {np.std(values):.2f}")

    df_result.to_excel(f"metricas_por_fold_{model_name}.xlsx", index=False)
    print(f"📊 Métricas por fold salvas em metricas_por_fold_{model_name}.xlsx")
    return all_results

# --- Execução principal ---
df = pd.read_csv('all_animal_images.csv')
df['Group'] = df['ID Animal'].apply(lambda x: 'C1' if 'C1_' in x else 'C2')

final_results = []
for model_name in model_info.keys():
    print(f"\nIniciando processamento com modelo: {model_name}")
    start_time = time.time()
    model = model_info[model_name][0](weights='imagenet', include_top=False, pooling='avg')
    results = run_stratified_kfold_regressions_from_images(df, model, model_name)
    final_results.extend(results)
    elapsed = time.time() - start_time
    print(f"Tempo total para {model_name}: {elapsed/60:.2f} minutos")

results_df = pd.DataFrame(final_results)
results_df.to_excel('regression_results_1024_DenseNet169_augmented_trainonly_stratified.xlsx', index=False)
print("Resultados salvos em 'regression_results_1024_DenseNet169_augmented_trainonly_stratified.xlsx'.")


Iniciando processamento com modelo: DenseNet169


Fold 1 - Teste (DenseNet169): 100%|████████████████████████████████████████████████████| 61/61 [00:24<00:00,  2.46it/s]



--- Regressão: Linear Regression | Modelo CNN: DenseNet169 ---
[Fold 1] MSE: 13.60 | RMSE: 3.69 | MAE: 2.73 | R2: 0.89 | R: 0.96

--- Regressão: Random Forest | Modelo CNN: DenseNet169 ---
[Fold 1] MSE: 28.53 | RMSE: 5.34 | MAE: 3.94 | R2: 0.77 | R: 0.89

--- Regressão: SVR | Modelo CNN: DenseNet169 ---
[Fold 1] MSE: 115.03 | RMSE: 10.73 | MAE: 8.76 | R2: 0.08 | R: 0.80

--- Regressão: Lasso | Modelo CNN: DenseNet169 ---
[Fold 1] MSE: 66.65 | RMSE: 8.16 | MAE: 6.42 | R2: 0.47 | R: 0.78

--- Regressão: ElasticNet | Modelo CNN: DenseNet169 ---
[Fold 1] MSE: 74.32 | RMSE: 8.62 | MAE: 6.75 | R2: 0.41 | R: 0.78

--- Regressão: Ridge | Modelo CNN: DenseNet169 ---
[Fold 1] MSE: 9.46 | RMSE: 3.08 | MAE: 2.44 | R2: 0.92 | R: 0.97


Fold 2 - Teste (DenseNet169): 100%|████████████████████████████████████████████████████| 71/71 [00:26<00:00,  2.73it/s]



--- Regressão: Linear Regression | Modelo CNN: DenseNet169 ---
[Fold 2] MSE: 20.27 | RMSE: 4.50 | MAE: 3.32 | R2: 0.84 | R: 0.96

--- Regressão: Random Forest | Modelo CNN: DenseNet169 ---
[Fold 2] MSE: 29.74 | RMSE: 5.45 | MAE: 4.49 | R2: 0.76 | R: 0.89

--- Regressão: SVR | Modelo CNN: DenseNet169 ---
[Fold 2] MSE: 109.47 | RMSE: 10.46 | MAE: 9.02 | R2: 0.13 | R: 0.75

--- Regressão: Lasso | Modelo CNN: DenseNet169 ---
[Fold 2] MSE: 72.77 | RMSE: 8.53 | MAE: 6.76 | R2: 0.42 | R: 0.76

--- Regressão: ElasticNet | Modelo CNN: DenseNet169 ---
[Fold 2] MSE: 79.43 | RMSE: 8.91 | MAE: 7.00 | R2: 0.37 | R: 0.76

--- Regressão: Ridge | Modelo CNN: DenseNet169 ---
[Fold 2] MSE: 9.72 | RMSE: 3.12 | MAE: 2.35 | R2: 0.92 | R: 0.97


...


--- Regressão: Linear Regression | Modelo CNN: DenseNet169 ---
[Fold 3] MSE: 13.34 | RMSE: 3.65 | MAE: 2.68 | R2: 0.87 | R: 0.94

--- Regressão: Random Forest | Modelo CNN: DenseNet169 ---
[Fold 3] MSE: 19.71 | RMSE: 4.44 | MAE: 3.54 | R2: 0.81 | R: 0.91

--- Regressão: SVR | Modelo CNN: DenseNet169 ---
[Fold 3] MSE: 89.05 | RMSE: 9.44 | MAE: 8.13 | R2: 0.13 | R: 0.75

--- Regressão: Lasso | Modelo CNN: DenseNet169 ---
[Fold 3] MSE: 90.68 | RMSE: 9.52 | MAE: 7.76 | R2: 0.12 | R: 0.75

--- Regressão: ElasticNet | Modelo CNN: DenseNet169 ---
[Fold 3] MSE: 87.61 | RMSE: 9.36 | MAE: 7.59 | R2: 0.15 | R: 0.75

--- Regressão: Ridge | Modelo CNN: DenseNet169 ---
[Fold 3] MSE: 12.32 | RMSE: 3.51 | MAE: 2.69 | R2: 0.88 | R: 0.94


Fold 4 - Teste (DenseNet169): 100%|████████████████████████████████████████████████████| 71/71 [00:40<00:00,  1.73it/s]



--- Regressão: Linear Regression | Modelo CNN: DenseNet169 ---
[Fold 4] MSE: 18.96 | RMSE: 4.35 | MAE: 3.14 | R2: 0.85 | R: 0.94

--- Regressão: Random Forest | Modelo CNN: DenseNet169 ---
[Fold 4] MSE: 34.95 | RMSE: 5.91 | MAE: 4.04 | R2: 0.72 | R: 0.86

--- Regressão: SVR | Modelo CNN: DenseNet169 ---
[Fold 4] MSE: 115.06 | RMSE: 10.73 | MAE: 8.97 | R2: 0.09 | R: 0.67

--- Regressão: Lasso | Modelo CNN: DenseNet169 ---
[Fold 4] MSE: 118.05 | RMSE: 10.87 | MAE: 8.51 | R2: 0.07 | R: 0.71

--- Regressão: ElasticNet | Modelo CNN: DenseNet169 ---
[Fold 4] MSE: 128.79 | RMSE: 11.35 | MAE: 8.90 | R2: -0.01 | R: 0.68

--- Regressão: Ridge | Modelo CNN: DenseNet169 ---
[Fold 4] MSE: 15.43 | RMSE: 3.93 | MAE: 2.89 | R2: 0.88 | R: 0.95


Fold 5 - Teste (DenseNet169): 100%|████████████████████████████████████████████████████| 65/65 [00:50<00:00,  1.29it/s]



--- Regressão: Linear Regression | Modelo CNN: DenseNet169 ---
[Fold 5] MSE: 18.57 | RMSE: 4.31 | MAE: 3.23 | R2: 0.82 | R: 0.94

--- Regressão: Random Forest | Modelo CNN: DenseNet169 ---
[Fold 5] MSE: 37.92 | RMSE: 6.16 | MAE: 4.43 | R2: 0.64 | R: 0.86

--- Regressão: SVR | Modelo CNN: DenseNet169 ---
[Fold 5] MSE: 96.63 | RMSE: 9.83 | MAE: 8.32 | R2: 0.08 | R: 0.58

--- Regressão: Lasso | Modelo CNN: DenseNet169 ---
[Fold 5] MSE: 103.59 | RMSE: 10.18 | MAE: 7.78 | R2: 0.01 | R: 0.59

--- Regressão: ElasticNet | Modelo CNN: DenseNet169 ---
[Fold 5] MSE: 105.54 | RMSE: 10.27 | MAE: 7.92 | R2: -0.01 | R: 0.58

--- Regressão: Ridge | Modelo CNN: DenseNet169 ---
[Fold 5] MSE: 13.30 | RMSE: 3.65 | MAE: 2.63 | R2: 0.87 | R: 0.94

📝 Predições consolidadas salvas em predicoes_completas_DenseNet169.txt

>>> MÉDIA DOS RESULTADOS - Linear Regression
MSE: 16.95 ± 2.90
RMSE: 4.10 ± 0.36
MAE: 3.02 ± 0.27
R2: 0.85 ± 0.02
R: 0.95 ± 0.01

>>> MÉDIA DOS RESULTADOS - Random Forest
MSE: 30.17 ± 6.25
RM